In [22]:
#compute all rouges between answer and passage

import glob
import pandas as pd
from rouge_score import rouge_scorer

rouge = rouge_scorer.RougeScorer(rouge_types=['rougeLsum','rougeL','rouge1','rouge2'], split_summaries=False)

final_files = glob.glob("/dccstor/srosent2/generative/appen/final/*/*/*_answerable.jsonl*")
all_scores = []

for final_file in final_files:
    data = pd.read_json(final_file, orient='records', lines=True, dtype={'id':str})

    for i, row in data.iterrows():
        for answer in row['output']:
            if 'answer' not in answer or answer['answer'] is None:
                continue
            scores = rouge.score(row['passages'][0]['text'], answer['answer'])
            all_scores.append(scores)

all_rouge_scores_df = pd.DataFrame(all_scores)
all_rouge_scores_df.head(4)

,rougeLsum,rougeL,rouge1,rouge2
0,"(0.7741935483870968, 0.15584415584415584, 0.25...","(0.7741935483870968, 0.15584415584415584, 0.25...","(0.8064516129032258, 0.16233766233766234, 0.27...","(0.5666666666666667, 0.1111111111111111, 0.185..."
1,"(0.9558823529411765, 0.4140127388535032, 0.577...","(0.9558823529411765, 0.4140127388535032, 0.577...","(1.0, 0.43312101910828027, 0.6044444444444445)","(0.8955223880597015, 0.38461538461538464, 0.53..."
2,"(0.76, 0.3630573248407643, 0.4913793103448276)","(0.76, 0.3630573248407643, 0.4913793103448276)","(0.92, 0.4394904458598726, 0.5948275862068966)","(0.7972972972972973, 0.3782051282051282, 0.513..."
3,"(0.9879518072289156, 0.36123348017621143, 0.52...","(0.9879518072289156, 0.36123348017621143, 0.52...","(0.9879518072289156, 0.36123348017621143, 0.52...","(0.8902439024390244, 0.3230088495575221, 0.474..."


In [24]:
metric = ['p', 'r', 'f']
for column_name in all_rouge_scores_df.columns:
    print(column_name)
    for index in range(3):
        print(f'{metric[index]}: {sum(row[index] for row in all_rouge_scores_df[column_name]) / len(all_rouge_scores_df)}')


rougeLsum
p: 0.9413969109652969
r: 0.3302728784728195
f: 0.4676396580763118
rougeL
p: 0.9411752427294209
r: 0.33021463647552174
f: 0.4675518255035
rouge1
p: 0.9751045527388809
r: 0.33982641820332193
f: 0.481781416791356
rouge2
p: 0.9010569308664522
r: 0.3149052955815063
f: 0.4460828751238865


In [6]:
# count how many flan-t5-xxl predictions are short answers.

import pandas as pd

longNQAnswerable = pd.read_json("/dccstor/srosent1/synthetic-question-generation/experiments/longNQ/flan-t5-xxl/longNQ-answerable-zs.jsonl", lines=True, orient='records')

longNQAnswerable.iloc[0]

id                                          6401197308716205056
input         which method of forecasting uses averages to p...
passages      [{'title': 'Forecasting', 'text': 'Seasonality...
output        [{'answer': 'Seasonality, in which data experi...
generation    unanswerable, unanswerable, unanswerable, unan...
Name: 0, dtype: object

In [19]:
short_answer = 0
for i, row in longNQAnswerable.iterrows():
    if len(row['generation'].split()) <= 10 and 'unanswerable' not in row['generation']: # or 'unanswerable' in row['generation']:
        print(f"Question: {row['input']}")
        if len(row['output']) == 2:
            print(f"Gold Answer: {row['output'][1]['answer']}")
        else:
            print(f"Gold Answer: {row['output'][0]['answer']}")
        print(f"Generated Answer: {row['generation']}")
        print("----")
        short_answer += 1
print(short_answer)

Question: where does the bronx get its name from
Gold Answer: The Bronx gets its name from the person of Jonas Bronck ( c. 1600 -- 43 ) who was the first recorded European settler in the area.
Generated Answer: Bronck 's River or the Bronx ( River )
----
Question: who is given credit for inventing the printing press
Gold Answer: The world 's first movable type printing press technology for printing paper books was made of porcelain materials and was invented around AD 1040 in China during the Northern Song Dynasty by the inventor Bi Sheng ( 990 -- 1051 ) .
Around 1450 Johannes Gutenberg , introduced the metal movable - type printing press in Europe.
Generated Answer: Bi Sheng ( 990 -- 1051 ) . Subsequently
----
Question: who does santana end up with in glee
Gold Answer: After performing in "Hand In My Pocket" / "I Feel the Earth Move" with Brittany, Santana Lopez proposed and Brittany accepted. They wed in a double ceremony with Kurt and Blaine in Glee's eighth episode A Wedding.
Gener

In [18]:
short_answer = 0
for i, row in longNQAnswerable.iterrows():
    if 'unanswerable' in row['generation']: # or 'unanswerable' in row['generation']:
        print(f"Question: {row['input']}")
        print(f"Gold Answer: {row['output'][0]['answer']}")
        print(f"Generated Answer: {row['generation']}")
        print("----")
        short_answer += 1
print(short_answer)

Question: which method of forecasting uses averages to predict future weather
Gold Answer: Seasonality, in which data experiences regular and predictable changes which recur every calendar year, is the method of forecasting, where averages are used to calculate the seasonal index of the season.
Generated Answer: unanswerable, unanswerable, unanswerable, unanswerable
----
Question: who has played chad dimera on days of our lives
Gold Answer: The role was originated by Casey Jon Deidrick on June 19, 2009.  Billy Flynn currently portrays the role, making his first appearance on September 12, 2014.
Generated Answer: Casey Jon Deidrick, Billy Flynn, unanswerable
----
Question: what is meant by sanctification in the bible
Gold Answer: Sanctification is the act or process of acquiring sanctity , of being made or becoming holy .
To sanctify is to literally `` set apart for particular use in a special purpose or work and to make holy or sacred . ''
Generated Answer: unanswerable; Sanctification

In [1]:
import glob
import pandas as pd
from rouge_score import rouge_scorer

rouge = rouge_scorer.RougeScorer(rouge_types=['rougeLsum','rougeL','rouge1','rouge2'], split_summaries=False)

result_files = glob.glob("/dccstor/srosent1/synthetic-question-generation/experiments/longNQ/*/*.jsonl")       

for result_file in result_files:
    avg_rouge_score = 0
    data = pd.read_json(result_file, lines=True, orient='records')
    max_score = -1
    for i, row in data.iterrows():
        for answer in row['output']:
            if 'answer' not in answer or answer['answer'] == None:
                if 'unanswerable' in row["generation"]:
                    score = 1
                else:
                    score = 0
            else:
                score = rouge.score(answer['answer'], row["generation"])['rougeLsum'][2]
            if score > max_score:
                    max_score = score
        avg_rouge_score += max_score
    print(f"{result_file},{avg_rouge_score/len(data)}")

/dccstor/srosent1/synthetic-question-generation/experiments/longNQ/flan-ul2/longNQ-longNQ_dev_unanswerable.jsonl-2s.jsonl,0.9966666666666667
/dccstor/srosent1/synthetic-question-generation/experiments/longNQ/flan-ul2/longNQ-longNQ_dev_answerable.jsonl-zs.jsonl,1.5410185138780157
/dccstor/srosent1/synthetic-question-generation/experiments/longNQ/flan-ul2/longNQ-longNQ_dev_answerable.jsonl-2s.jsonl,1.563651757004096
/dccstor/srosent1/synthetic-question-generation/experiments/longNQ/flan-ul2/longNQ-longNQ_dev_unanswerable.jsonl-zs.jsonl,0.9966666666666667
/dccstor/srosent1/synthetic-question-generation/experiments/longNQ/falcon-40b/longNQ-longNQ_dev_unanswerable.jsonl-2s.jsonl,0.9966666666666667
/dccstor/srosent1/synthetic-question-generation/experiments/longNQ/falcon-40b/longNQ-longNQ_dev_unanswerable.jsonl-zs.jsonl,0.9966666666666667
/dccstor/srosent1/synthetic-question-generation/experiments/longNQ/llama-2-70b/longNQ-longNQ_dev_unanswerable.jsonl-2s.jsonl,1.0
/dccstor/srosent1/syntheti